# DESCRIÇÃO DO NEGÓCIO

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

Submissions are evaluated on the Root Mean Square Percentage Error.

## Initial Parameters

In [1]:
show_graphics = False

run_boruta = False

fine_tunning = False

# store_fraction reduces the number of stores to train n times
store_fraction = 17


# 0.0 IMPORTS


In [2]:
from IPython.display       import Image
from IPython.core.display  import HTML
import warnings

import pandas as pd
import inflection # biblioteca para padronizar o nome das variaveis
import math
import numpy as np
import scipy.stats as ss

from matplotlib            import pyplot as plt
import seaborn as sns

import pickle

from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import RandomizedSearchCV


import xgboost as xgb

if(run_boruta):
    from boruta import BorutaPy

warnings.filterwarnings( 'ignore' )

## 0.1 Helper Functions

In [3]:
def cramer_v(x, y):
    cm = pd.crosstab( x, y).values
    chi2 = ss.chi2_contingency( cm )[0]
    n = cm.sum()
    r,k = cm.shape
    
    chi2corr = max(0, chi2 -(k-1)*(r-1)/(n-1))
    kcorr = k - (k-1)**2/(n-1)
    rcorr = r - (r-1)**2/(n-1)
        
    return np.sqrt( (chi2corr/n) / (min(kcorr-1,rcorr-1) ) )

def mean_percentage_error( y, yhat ):
    return np.mean( ( y - yhat ) / y )
     
    
def mean_absolute_percentage_error( y, yhat ):
    return np.mean( np.abs( ( y - yhat ) / y ) )

    
def ml_error( model_name, y, yhat ):
    mae = mean_absolute_error( y, yhat )
    mape = mean_absolute_percentage_error( y, yhat )
    rmse = np.sqrt( mean_squared_error( y, yhat ) )
    
    return pd.DataFrame( { 'Model Name': model_name, 
                           'MAE': mae, 
                           'MAPE': mape,
                           'RMSE': rmse }, index=[0] )

def cross_validation( X, kfold, model_name, model, verbose=False ):
    mae_list = []
    mape_list = []
    rmse_list = []
    for k in reversed( range( 1, kfold+1 ) ):
        if verbose:
            print( '\nKFold Number: {}'.format( k ) )
            
        # start and end date for validation 
        validation_start_date = X['date'].max() - datetime.timedelta( days=k*6*7)
        validation_end_date = X['date'].max() - datetime.timedelta( days=(k-1)*6*7)

        # Subsetting dates
        X_train = X[X['date'] < validation_start_date]
        X_validation = X[(X['date'] >= validation_start_date) & (X['date'] <= validation_end_date)]

        # training and validation dataset
        # training
        y_train = X_train['sales']
        X_train = X_train.drop( ['date', 'sales'], axis=1 ) 
        

        # validation
        y_validation = X_validation['sales']
        X_validation = X_validation.drop( ['date', 'sales'], axis=1 )
        
        # model
        model.fit( X_train, y_train )

        # prediction
        yhat = model.predict( X_validation )

        # performance
        m_result = ml_error( model_name, np.expm1( y_validation ), np.expm1( yhat ) )

        # store performance of each kfold iteration
        mae_list.append(  m_result['MAE'] )
        mape_list.append( m_result['MAPE'] )
        rmse_list.append( m_result['RMSE'] )

    return pd.DataFrame( {'Model Name': model_name,
                          'MAE CV': np.round( np.mean( mae_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mae_list ), 2 ).astype( str ),
                          'MAPE CV': np.round( np.mean( mape_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mape_list ), 2 ).astype( str ),
                          'RMSE CV': np.round( np.mean( rmse_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( rmse_list ), 2 ).astype( str ) }, index=[0] )

def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [4]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 0.2 Loading data

In [12]:
df_sales_raw = pd.read_csv("data/train.csv", low_memory=False)
df_store_raw = pd.read_csv("data/store.csv", low_memory=False)

# merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')


In [6]:
df_raw.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


# 1.0 DATA DESCRIPTION

Data fields
Most of the fields are self-explanatory. The following are descriptions for those that aren't.

* Id - an Id that represents a (Store, Date) duple within the test set
* Store - a unique Id for each store
* Sales - the turnover for any given day (this is what you are predicting)
* Customers - the number of customers on a given day
* Open - an indicator for whether the store was open: 0 = closed, 1 = open
* StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* StoreType - differentiates between 4 different store models: a, b, c, d
* Assortment - describes an assortment level: a = basic, b = extra, c = extended
* CompetitionDistance - distance in meters to the nearest competitor store
* CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* Promo - indicates whether a store is running a promo on that day
* Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [7]:
df1 = df_raw[df_raw['Store'] % store_fraction ==0].copy()

## 1.1 Rename Columns

In [8]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase, cols_old))

# rename
df1.columns = cols_new



## 1.2 Data Dimensions

In [9]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Cols: {}'.format(df1.shape[1]))

Number of Rows: 59574
Number of Cols: 18


## 1.3 Data Types

In [10]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [11]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4 Check NA

Há 3 formas de substituir NAs
* Drop NA rows
* Using statistics and machine learning to infere data
* Understanding why the data is missing

In [12]:
df1.isna().sum()

store                               0
day_of_week                         0
date                                0
sales                               0
customers                           0
open                                0
promo                               0
state_holiday                       0
school_holiday                      0
store_type                          0
assortment                          0
competition_distance                0
competition_open_since_month    16772
competition_open_since_year     16772
promo2                              0
promo2_since_week               25824
promo2_since_year               25824
promo_interval                  25824
dtype: int64

## 1.5 Fillout NA

In [13]:
#competition_distance              2642
# Imaginar que não há competidores próximos, ou seja, distância grande.
faraway = 100*df1.competition_distance.max()
near = 50

df1['competition_distance'].fillna(faraway, inplace=True)
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: near if x <=near else x)

# A ideia aqui é aplicar o data atual (do dado lido) para dizer depois que não há tempo com concorrente.
# applicar df1.date para os campos de competition open since

#competition_open_since_month    323348
df1['competition_open_since_month']= df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

#competition_open_since_year     323348
df1['competition_open_since_year']= df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)


In [15]:
# Para os dados de promo2_since_, fazer o mesmo. Aparentemente, não os dados faltantes são os que não há promo2,
# checar na mão!

print('Number of rows with promo2 == 0: {}'.format(df1[df1['promo2']== 0].shape[0]))
print('Number of rows with promo2 == 1 and missing since values: {}'.format((df1[df1['promo2']== 1 & (df1['promo2_since_week'].isna() | df1['promo2_since_year'].isna())].shape[0])))
      

Number of rows with promo2 == 0: 25824
Number of rows with promo2 == 1 and missing since values: 0


In [16]:
# Atribuir a data atual (da leitura) para dizer que não há tempo desde a última promo.
#promo2_since_week              508031

df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

#promo2_since_year               508031
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)


#promo_interval  

# criar month map dict
month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

# criar colunas 'month_map' com base em date
df1['month_map'] = df1['date'].dt.month.map(month_map)

# criar 'is_promo' comparando o map com promo_inverval
df1['promo_interval'].fillna('', inplace=True)


df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 1 if x['month_map'] in x['promo_interval'].split(',')
                            else 0 ,axis=1)



## 1.6. Change Data Types


In [18]:
# competiton
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype( int64 )
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype( int64 )
    
# promo2
df1['promo2_since_week'] = df1['promo2_since_week'].astype( int64 )
df1['promo2_since_year'] = df1['promo2_since_year'].astype( int64 )

## 1.7. Descriptive Statistics

In [19]:
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month             int64
competition_open_since_year              int64
promo2                                   int64
promo2_since_week                        int64
promo2_since_year                        int64
promo_interval                          object
month_map                               object
is_promo                                 int64
dtype: object

In [20]:
num_attributes = df1.select_dtypes( include=['int64', 'float64'] )
cat_attributes = df1.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'] )

num_attributes_list = list(num_attributes.columns)
cat_attributes_list = list(cat_attributes.columns)

In [21]:
num_attributes.head()

,store,day_of_week,sales,customers,open,promo,school_holiday,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,is_promo
16,17,5,8430,946,1,1,1,50.0,12,2005,1,26,2010,1
33,34,5,11144,1162,1,1,1,2240.0,9,2009,0,31,2015,0
50,51,5,9198,610,1,1,0,10570.0,7,2013,1,9,2011,1
67,68,5,11187,1341,1,1,1,250.0,7,2015,1,35,2012,0
84,85,5,7791,971,1,1,1,1870.0,10,2011,0,31,2015,0


In [22]:
cat_attributes.head()

,state_holiday,store_type,assortment,promo_interval,month_map
16,0,a,a,"Jan,Apr,Jul,Oct",Jul
33,0,c,a,,Jul
50,0,a,c,"Jan,Apr,Jul,Oct",Jul
67,0,a,c,"Mar,Jun,Sept,Dec",Jul
84,0,b,a,,Jul


### 1.7.1. Numerical Atributes

Aqui parece ser mais razoável selecionar as colunas que façam sentido ter estatísticas.
Algumas variáveis aqui são categóricas mas o encoding já está feito como:
- store
- day_of_week
- promo2

In [23]:
# Central Tendency - mean, median 

m = num_attributes.describe()
#ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
#ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# dispersion - std, min, max, range, skew, kurtosis
#d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T 
#d2 = pd.DataFrame( num_attributes.apply( min ) ).T 
#d3 = pd.DataFrame( num_attributes.apply( max ) ).T 
#d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T 
#d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T 
#d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T 

# concatenar
#m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
#m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
#m
#ct1
m

,store,day_of_week,sales,customers,open,promo,school_holiday,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,is_promo
count,59574.000000,59574.000000,59574.000000,59574.000000,59574.000000,59574.000000,59574.000000,59574.000000,59574.000000,59574.000000,59574.000000,59574.000000,59574.000000,59574.000000
mean,562.942727,3.998271,5626.912428,614.360056,0.832209,0.381609,0.175060,5684.942760,7.216621,2010.226407,0.566522,24.030164,2012.690905,0.186994
std,318.996803,1.997400,3630.477237,403.144225,0.373684,0.485786,0.380021,6549.599528,3.287028,4.615929,0.495559,13.936008,1.673691,0.389910
min,17.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,1.000000,1999.000000,0.000000,1.000000,2009.000000,0.000000
25%,289.000000,2.000000,3838.000000,416.000000,1.000000,0.000000,0.000000,620.000000,4.000000,2007.000000,0.000000,12.000000,2012.000000,0.000000
50%,561.000000,4.000000,5633.000000,609.000000,1.000000,0.000000,0.000000,2490.000000,7.000000,2012.000000,1.000000,23.000000,2013.000000,0.000000
75%,833.000000,6.000000,7581.750000,832.000000,1.000000,1.000000,0.000000,8260.000000,10.000000,2014.000000,1.000000,35.000000,2014.000000,0.000000
max,1105.000000,7.000000,31415.000000,3145.000000,1.000000,1.000000,1.000000,25430.000000,12.000000,2015.000000,1.000000,52.000000,2015.000000,1.000000


### 1.7.2. Categorical Atributes

In [24]:
cat_attributes.apply( lambda x: x.unique().shape[0] )


state_holiday      4
store_type         4
assortment         2
promo_interval     4
month_map         12
dtype: int64

In [25]:
if(show_graphics):
    
    aux = df1[(df1['state_holiday'] != '0') & (df1['sales'] > 0)]

    plt.subplot( 1, 3, 1 )
    sns.boxplot( x='state_holiday', y='sales', data=aux )

    plt.subplot( 1, 3, 2 )
    sns.boxplot( x='store_type', y='sales', data=aux )

    plt.subplot( 1, 3, 3 )
    sns.boxplot( x='assortment', y='sales', data=aux )

# 2.0. FEATURE ENGINEERING

In [26]:
df2 = df1.copy()

## 2.1. Hypothesys Mindmap

Fazer um mapa mental com as hipóteses a serem levantadas.

## 2.2. Criacao das Hipoteses


Registro da lista de hipóteses a serem avaliadas com brainstorming.

## 2.3. Lista Final de Hipóteses


Apenas escolher algumas hipóteses aparentemente mais relevantes.

## 2.4. Feature Engineering

Derivar algumas variáveis para facilitar a avaliação

In [27]:
# Derivando datas como variáveis categóricas
# year
df2['year'] = df2['date'].dt.year

# month
df2['month'] = df2['date'].dt.month

# day
df2['day'] = df2['date'].dt.day

# week of year
df2['week_of_year'] = df2['date'].dt.weekofyear

# year week
df2['year_week'] = df2['date'].dt.strftime( '%Y-%W' )

In [28]:
# competition since

# criando uma única feature para dizer a data de início da promoção
df2['competition_since'] = df2.apply( lambda x: datetime.datetime( year=x['competition_open_since_year'], month=x['competition_open_since_month'],day=1 ), axis=1 )

# calcula o timedelta do início da promoção para a data do registro
df2['competition_time_month'] = ( ( df2['date'] - df2['competition_since'] )/30 ).apply( lambda x: x.days ).astype( int64 )

In [29]:
# promo since

# cria uma str com o formato da data
df2['promo_since'] = df2['promo2_since_year'].astype( str ) + '-' + df2['promo2_since_week'].astype( str )

# transforma a str em data
df2['promo_since'] = df2['promo_since'].apply( lambda x: datetime.datetime.strptime( x + '-1', '%Y-%W-%w' ) - datetime.timedelta( days=7 ) )

# calcula o timedelta da promoção em semanas
df2['promo_time_week'] = ( ( df2['date'] - df2['promo_since'] )/7 ).apply( lambda x: x.days ).astype( int64 )



In [30]:
# competition strenght - indica de forma mais direta a interferência da competição

df2['competition_str'] = 1/df2['competition_distance']


# PASSO 03 - FILTRAGEM DE VARIÁVEIS


In [31]:
df3 = df2.copy()

## Filtragem das Linhas


In [32]:
# Exclui linhas quando as lojas estavam fechadas ou não houve vendas
df3 = df3[(df3['open'] != 0) & (df3['sales'] > 0)]

## Feature Selection

In [33]:
#Excluindo features que não fazem sentido para modelagem
cols_drop = ['customers', 'open', 'promo_interval', 'month_map', 'competition_open_since_year', 'competition_open_since_month', 'competition_since', 'promo_since', 'promo2_since_year', 'promo2_since_week', 'competition_distance' ]
df3 = df3.drop( cols_drop, axis=1 )
df3.columns

Index(['store', 'day_of_week', 'date', 'sales', 'promo', 'state_holiday',
       'school_holiday', 'store_type', 'assortment', 'promo2', 'is_promo',
       'year', 'month', 'day', 'week_of_year', 'year_week',
       'competition_time_month', 'promo_time_week', 'competition_str'],
      dtype='object')

In [34]:
num_attributes_list = ['competition_str',
                       'promo_time_week',
                       'competition_time_month']
cat_attributes_list = ['store',
                       'day_of_week',
                       'promo',
                       'state_holiday',
                       'school_holiday',
                       'store_type',
                       'assortment',
                       'is_promo',
                       'day',
                       'month',
                       'week_of_year',
                       'year_week']

# EXPLORATORY DATA ANALYSIS

In [35]:
df4 = df3.copy()

## Univariate Analysis

É interessante que as variáveis tenham distribuição normal para melhor performance dos modelos de ML.

### Numercial Features

In [36]:
if(show_graphics):
    for i, var in enumerate(num_attributes_list):
        plt.subplot(3, 1, i+1)
        sns.histplot(df4[var])

### Target Variable - sales

In [37]:
if(show_graphics):
    sns.distplot(df4['sales'])

### Numerical Features vs Target

In [38]:
if(show_graphics):

    for i , attribute in enumerate(num_attributes_list):
        plt.subplot(2, 2, i+1)
        sns.regplot(attribute, 'sales', data = df4[['sales', attribute]], line_kws={'color': 'r'})

### Categorical Features

In [39]:
if(show_graphics):

    for i , attribute in enumerate(cat_attributes_list):
        aux = df4[['sales', attribute]].groupby(attribute).mean().reset_index()
        plt.subplot(4, 3, i+1)
        sns.barplot(x = attribute, y= 'sales', data = aux)

        #sns.regplot(attribute, 'sales', data = num_attributes)

## Multivariate Analysis


### Numerical Attributes

In [40]:
if(show_graphics):

    num_correlation = df4[num_attributes_list].corr(method = 'pearson')
    sns.heatmap(num_correlation, annot = True)

### Categorical Attributes

In [41]:
aux_list = ['state_holiday', 'store_type', 'assortment']
aux_dict = {}

for var1 in aux_list:
    a_list = []
    for var2 in aux_list:
        a = cramer_v(df4[var1], df4[var2])
        a_list.append(a)
    aux_dict[var1] = a_list
     
aux_df = pd.DataFrame(aux_dict)
aux_df = aux_df.set_index(aux_df.columns)

In [42]:
if(show_graphics):
    sns.heatmap( aux_df, annot = True)

# Data Preparation

In [43]:
df5 = df4.copy()

## Normalization

A normalizção cabe para variáveis com distribuição normal, mas, aparentemente, não é o caso de nenhuma variável.

## Rescaling

In [80]:

# competition_str
rs = RobustScaler()
mms = MinMaxScaler()

df5['competition_str'] = rs.fit_transform(df5[['competition_str']])
pickle.dump(rs, open('parameter/encoding_competition_str.pkl', 'wb'))

# competition time_month
df5['competition_time_month'] = mms.fit_transform(df5[['competition_time_month']])
pickle.dump(mms, open('parameter/encoding_competition_time_month.pkl', 'wb'))

# promo_time_week
df5['promo_time_week'] = mms.fit_transform(df5[['promo_time_week']])
pickle.dump(mms, open('parameter/encoding_promo_time_week.pkl', 'wb'))

# year
df5['year'] = mms.fit_transform(df5[['year']])
pickle.dump(mms, open('parameter/encoding_year.pkl', 'wb'))



## Encoding

In [45]:
# state_holiday - One Hot Encoding
df5 = pd.get_dummies(df5, prefix=['state_holiday'], columns=['state_holiday'])

# store_type - Label Encoding
le = LabelEncoder()
df5['store_type'] = le.fit_transform(df5['store_type'])
pickle.dump(le, open('parameter/encoding_store_type.pkl', 'wb'))


# assortment - Ordinal Encoding
assortment_dict = {'a':1,'b':2,'c':3}
df5['assortment'] = df5['assortment'].map(assortment_dict)

## Transformations

### Response variable transformation

Log is applyed to 'sales' to better fit the variable distribuition to a normal curve.

In [46]:
df5['sales'] = np.log1p(df5['sales'])

### Nature transformation for cyclical variables

In [47]:
# day of week
df5['day_of_week_sin'] = df5['day_of_week'].apply(lambda x : np.sin(x*(2.*np.pi/7)))
df5['day_of_week_cos'] = df5['day_of_week'].apply(lambda x : np.cos(x*(2.*np.pi/7)))

# month
df5['month_sin'] = df5['month'].apply(lambda x : np.sin(x*(2.*np.pi/12)))
df5['month_cos'] = df5['month'].apply(lambda x : np.cos(x*(2.*np.pi/12)))

# day
df5['day_sin'] = df5['day'].apply(lambda x : np.sin(x*(2.*np.pi/30)))
df5['day_cos'] = df5['day'].apply(lambda x : np.cos(x*(2.*np.pi/30)))

# week of year
df5['week_of_year_sin'] = df5['week_of_year'].apply(lambda x : np.sin(x*(2.*np.pi/52)))
df5['week_of_year_cos'] = df5['week_of_year'].apply(lambda x : np.cos(x*(2.*np.pi/52)))


# Feature Selection

* Seleção univariada utiliza a correlação para selecionar as variáveis
* Seleção por importância treina modelos como random forest ou regression linear (verifica coeficientes) para escolher as variáveis
* Seleção por Subset:
    * Selecione uma variável aleaoriamente, treine um modelo (random forest ou xgboost) e calcule a performance
    * Loop: Adicione outra variável e verifique se a perfomance aumentou. Se sim, manter a variável.
    * O Boruta funciona dessa forma, mas comparando com a melhor variável shadow e usando o p-value.
    


In [48]:
df6 = df5.copy()

## Train-Test Split

In [49]:
df6.columns

Index(['store', 'day_of_week', 'date', 'sales', 'promo', 'school_holiday',
       'store_type', 'assortment', 'promo2', 'is_promo', 'year', 'month',
       'day', 'week_of_year', 'year_week', 'competition_time_month',
       'promo_time_week', 'competition_str', 'state_holiday_0',
       'state_holiday_a', 'state_holiday_b', 'state_holiday_c',
       'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos',
       'day_sin', 'day_cos', 'week_of_year_sin', 'week_of_year_cos'],
      dtype='object')

In [50]:
cols_drop = ['week_of_year', 'day', 'month', 'day_of_week']
df6 = df6.drop(cols_drop, axis=1)

In [51]:
train_date_limit = df6[['store', 'date']].groupby('store').max().reset_index()['date'][0] - datetime.timedelta(days = 6*7)

X_train = df6[df6['date'] < train_date_limit ]
y_train = X_train['sales']


X_test = df6[df6['date'] >= train_date_limit ]
y_test = X_test['sales']

print( 'Train Min date: {}'.format(X_train['date'].min()))
print( 'Train Max date: {}'.format(X_train['date'].max()))
print('\n')
print( 'Test  Min date: {}'.format(X_test['date'].min()))
print( 'Test  Max date: {}'.format(X_test['date'].max()))

# não parece razoável dropar a data, mas como os dados da data estão contidos em outros atributos, ok
X_train_n = X_train.drop(['date', 'sales'], axis = 1).values
y_train_n = y_train.values.ravel()


Train Min date: 2013-01-01 00:00:00
Train Max date: 2015-06-18 00:00:00


Test  Min date: 2015-06-19 00:00:00
Test  Max date: 2015-07-31 00:00:00


## Boruta Feature Selection

In [52]:
if(run_boruta):
    
    # define RandomForestRegressor
    rf = RandomForestRegressor(n_jobs = -1)
    
    boruta = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=42).fit(X_train_n, y_train_n)
    
    cols_selected = boruta.support_.tolist()
    cols_selected_boruta = X
    

### Best Features from Boruta

In [53]:
if(run_boruta):
    cols_selected = boruta.support_.tolist()

    # best features
    X_train_fs = X_train.drop( ['date', 'sales'], axis=1 )
    cols_selected_boruta = X_train_fs.iloc[:, cols_selected].columns.to_list()

    # not selected boruta
    cols_not_selected_boruta = list( np.setdiff1d( X_train_fs.columns, cols_selected_boruta ) )
else:
    # Manual feature selection
    cols_selected_boruta = ['store',
                            'promo',
                            'store_type',
                            'assortment',
                            'promo2',
                            'competition_time_month',
                            'promo_time_week',
                            'competition_str',
                            'day_of_week_sin',
                            'day_of_week_cos',
                            'month_cos',
                            'day_sin',
                            'day_cos',
                            'week_of_year_cos'
                           ]
    
    cols_not_selected_boruta = ['is_promo', 'month_sin', 'school_holiday', 'state_holiday_0',
       'state_holiday_a', 'state_holiday_b', 'state_holiday_c', 'week_of_year_sin', 'year']


# columns to add
feat_to_add = ['date', 'sales']

cols_selected_boruta_full = cols_selected_boruta.copy()
cols_selected_boruta_full.extend( feat_to_add )

# Machine Learning Modeling

* Average Model - Modelo de referência
* Linear Regression
* Linear Regression Regularized (Lasso e Ridge)
* Random Forest Regressor
* XGBoost Regressor


In [54]:
X_train = X_train[cols_selected_boruta]
X_test  = X_test[cols_selected_boruta]

## Average Model

In [55]:
aux1 = X_test.copy()
aux1['sales'] = y_test.copy()

In [56]:
# prediction

aux2 = aux1[['store', 'sales']].groupby('store').mean().reset_index().rename(columns = {'sales' : 'predictions'})
aux1 = pd.merge(aux1, aux2, how = 'left', on='store')

yhat_baseline = aux1['predictions']

In [57]:
# performance

baseline_result = ml_error('Average Model', np.expm1(y_test), np.expm1(yhat_baseline))
baseline_result

,Model Name,MAE,MAPE,RMSE
0,Average Model,1308.743555,0.331512,1768.353952


## Linear Regression Model

In [58]:
# model
lr = LinearRegression()
lr.fit( X_train, y_train )

# prediction
yhat_lr = lr.predict( X_test )

# performance
lr_result = ml_error( 'Linear Regression', np.expm1( y_test ), np.expm1( yhat_lr ) )
lr_result

,Model Name,MAE,MAPE,RMSE
0,Linear Regression,1618.505794,0.248724,2363.620074


## Linear Regression Model Relularized

In [59]:
# model
lrr = Lasso(alpha = 0.01).fit( X_train, y_train )

# prediction
yhat_lrr = lrr.predict( X_test )

# performance
lrr_result = ml_error( 'Lasso', np.expm1( y_test ), np.expm1( yhat_lrr ) )
lrr_result

,Model Name,MAE,MAPE,RMSE
0,Lasso,1644.541822,0.250943,2407.794987


## Random Forest Regressor

In [60]:
# model
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42).fit(X_train, y_train)

# prediction
yhat_rf = rf.predict(X_test)

# performance
rf_result = ml_error('Random Forest', np.expm1(y_test), np.expm1(yhat_rf))
rf_result

,Model Name,MAE,MAPE,RMSE
0,Random Forest,711.614893,0.110912,991.766987


## XGBoost Regressor

In [61]:
# model
model_xgb = xgb.XGBRegressor(objective = 'reg:squarederror',
                             n_estimators=100,
                             eta = 0.01,
                             subsample = 0.7
                            ).fit(X_train, y_train)

# prediction
yhat_xgb = model_xgb.predict(X_test)

# performance
xgb_result = ml_error('XGBoost', np.expm1(y_test), np.expm1(yhat_xgb))
xgb_result

,Model Name,MAE,MAPE,RMSE
0,XGBoost,6427.166832,0.950293,6950.909755


## Model Performance Comparisson

In [62]:
modeling_result = pd.concat([baseline_result, lr_result, lrr_result, rf_result, xgb_result])
modeling_result.sort_values('RMSE')

,Model Name,MAE,MAPE,RMSE
0,Random Forest,711.614893,0.110912,991.766987
0,Average Model,1308.743555,0.331512,1768.353952
0,Linear Regression,1618.505794,0.248724,2363.620074
0,Lasso,1644.541822,0.250943,2407.794987
0,XGBoost,6427.166832,0.950293,6950.909755


## Cross Validation Performance Comparisson

In [63]:
X_train_full = df6[df6['date'] < train_date_limit ][cols_selected_boruta_full]

lr_result_cv = cross_validation(X_train_full, 5, 'Linear Regression', lr, verbose =  False)
lrr_result_cv = cross_validation(X_train_full, 5, 'Lasso', lrr, verbose =  False)
rf_result_cv = cross_validation(X_train_full, 5, 'Random Forest', rf, verbose =  False)
xgb_result_cv = cross_validation(X_train_full, 5, 'XGB', model_xgb, verbose =  False)    

In [64]:
modeling_result_cv = pd.concat([lr_result_cv, lrr_result_cv, rf_result_cv, xgb_result_cv])
modeling_result_cv.sort_values('RMSE CV')

,Model Name,MAE CV,MAPE CV,RMSE CV
0,Random Forest,800.6 +/- 176.87,0.11 +/- 0.02,1197.61 +/- 249.24
0,Linear Regression,1850.08 +/- 334.6,0.26 +/- 0.02,2724.08 +/- 524.9
0,Lasso,1875.99 +/- 370.17,0.26 +/- 0.01,2774.96 +/- 558.03
0,XGB,6811.76 +/- 626.91,0.95 +/- 0.0,7375.16 +/- 721.15


# Hyperparameter Fine Tuning

* Random Search - Define aleatoriamente os valores em uma tabela, define um número de interações e testa o melhor.
    * Método mais rápido mas impreciso.
* Grid Search - Procura todas as combinações possíveis
* Bayesian Search - Usa teoria de Bayes

## Random Search for Random Forest

In [65]:
random_grid  = {
    'n_estimators': [50, 80, 100, 150, 200, 350, 500, 900, 1300, 1700],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [66]:
if fine_tunning:
    # model
    rf = RandomForestRegressor()

    rf_random = RandomizedSearchCV(estimator = rf,
                                   param_distributions = random_grid,
                                   n_iter = 20,
                                   cv = 2,
                                   verbose=2,
                                   random_state=42,
                                   n_jobs = -1)


    rf_random.fit(X_train, y_train)

    print(rf_random.best_params_)
else:
    best_params = {'n_estimators': 350, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 100, 'bootstrap': False}
    rf_random = RandomForestRegressor(n_estimators = best_params['n_estimators'],
                                      min_samples_split = best_params['min_samples_split'],
                                      min_samples_leaf = best_params['min_samples_leaf'],
                                      max_features = best_params['max_features'],
                                      max_depth = best_params['max_depth'],
                                      bootstrap = best_params['bootstrap'],
                                      random_state=42,
                                      n_jobs = -1                                  
                                     ).fit(X_train, y_train)
        


In [67]:
    yhat_rf_hpt = rf_random.predict(X_test)

    rf_result = ml_error('Random Forest HPT', np.expm1(y_test), np.expm1(yhat_rf_hpt))
    rf_result

,Model Name,MAE,MAPE,RMSE
0,Random Forest HPT,688.928838,0.106288,983.855366


# Error Interpretation

In [68]:
df7 =  df6[df6['date'] >= train_date_limit ][cols_selected_boruta_full]

# rescale
df7['sales'] = np.expm1(df7['sales'])
df7['predictions'] = np.expm1(yhat_rf_hpt)


## Business Performance

In [69]:
# quantify sales in the entire period
# sum of predictions
df7_grouped = df7[['store', 'predictions']].groupby('store').sum().reset_index()

# MAE and MAPE
aux1 = df7[['store', 'sales', 'predictions']].groupby('store').apply(lambda x: mean_absolute_error(x['sales'], x['predictions'])).reset_index().rename(columns={0:'MAE'})
aux2 = df7[['store', 'sales', 'predictions']].groupby('store').apply(lambda x: mean_absolute_percentage_error(x['sales'], x['predictions'])).reset_index().rename(columns={0:'MAPE'})

aux3 = pd.merge(aux1, aux2, how = 'inner', on = 'store')
business = pd.merge(df7_grouped, aux3, how = 'inner', on = 'store')

# Scenarios
business['worst_scenario'] = business['predictions'] - aux3['MAE']
business['best_scenario'] = business['predictions'] + aux3['MAE']

# Reorder
business = business[['store', 'predictions', 'worst_scenario', 'best_scenario', 'MAE', 'MAPE' ]]

In [70]:
business.sort_values('MAPE', ascending = False).head()


,store,predictions,worst_scenario,best_scenario,MAE,MAPE
45,782,218157.982639,217291.656223,219024.309055,866.326416,0.237216
30,527,308846.144969,306422.040778,311270.249159,2424.104190,0.217538
9,170,180809.531363,179992.820798,181626.241927,816.710564,0.216254
8,153,257752.982330,256762.708798,258743.255862,990.273532,0.179910
63,1088,201804.905638,201003.168822,202606.642454,801.736816,0.169856


In [72]:
if show_graphics:
    sns.scatterplot(x= 'store', y = 'MAPE', data = business)

## Total Performance

In [73]:
business_total = business[['predictions', 'worst_scenario', 'best_scenario']].sum().reset_index().rename(columns = {'index': 'Scenario', 0: 'Total Sales'})
business_total['Total Sales'] = business_total['Total Sales'].map('R$ {:,.2}'.format)
business_total.head()

,Scenario,Total Sales
0,predictions,R$ 1.6e+07
1,worst_scenario,R$ 1.6e+07
2,best_scenario,R$ 1.6e+07


## Machine Learning Performance

In [74]:
df7['error'] = df7['sales'] - df7['predictions']
df7['error_rate'] = df7['error'] / df7['sales']

In [75]:
if show_graphics:

    plt.subplot(2, 2, 1)
    sns.lineplot(x='date', y='sales', data = df7, label = 'SALES')
    sns.lineplot(x='date', y='predictions', data = df7, label = 'PREDICTIONS')

    plt.subplot(2, 2, 2)
    sns.lineplot(x='date', y='error_rate', data = df7)

    plt.subplot(2, 2, 3)
    sns.distplot(df7['error'])

    plt.subplot(2, 2, 4)
    sns.scatterplot(df7['predictions'], df7['error'])

# Save model to production

In [77]:
import pickle

pickle.dump(rf_random, open(r'C:\Users\06564176686\repos\Store-Sales-Prediction\model\model_rossmann.pkl', 'wb'))

## Rossmann Class

In [3]:
import pickle
import inflection
import pandas as pd
import numpy as np
import math
import datetime


class Rossmann(object):
    def __init__(self):
        self.home_path = r'C:\Users\06564176686\repos\Store-Sales-Prediction'
        self.encoding_competition_str        = pickle.load(open(self.home_path + 'parameter\encoding_competition_str.pkl', 'rb'))
        self.encoding_competition_time_month = pickle.load(open(self.home_path + 'parameter\encoding_competition_time_month.pkl', 'rb'))
        self.encoding_promo_time_week        = pickle.load(open(self.home_path + 'parameter\encoding_promo_time_week.pkl', 'rb'))
        self.encoding_year                   = pickle.load(open(self.home_path + 'parameter\encoding_year.pkl', 'rb'))
        self.encoding_store_type             = pickle.load(open(self.home_path + 'parameter\encoding_store_type.pkl', 'rb'))


    def data_cleaning(self, df1):
        cols_old = ['Store', 'DayOfWeek', 'Date', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

        snakecase = lambda x: inflection.underscore(x)
        cols_new = list(map(snakecase, cols_old))
        
        # rename
        df1.columns = cols_new
        
        df1['date'] = pd.to_datetime(df1['date'])
        
        #competition_distance              2642
        # Imaginar que não há competidores próximos, ou seja, distância grande.
        faraway = 100*df1.competition_distance.max()
        near = 50

        df1['competition_distance'].fillna(faraway, inplace=True)
        df1['competition_distance'] = df1['competition_distance'].apply(lambda x: near if x <=near else x)

        # A ideia aqui é aplicar o data atual (do dado lido) para dizer depois que não há tempo com concorrente.
        # applicar df1.date para os campos de competition open since

        #competition_open_since_month    323348
        df1['competition_open_since_month']= df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

        #competition_open_since_year     323348
        df1['competition_open_since_year']= df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)
        
        
        # Atribuir a data atual (da leitura) para dizer que não há tempo desde a última promo.
        #promo2_since_week              508031

        df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

        #promo2_since_year               508031
        df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)


        #promo_interval  

        # criar month map dict
        month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

        # criar colunas 'month_map' com base em date
        df1['month_map'] = df1['date'].dt.month.map(month_map)

        # criar 'is_promo' comparando o map com promo_inverval
        df1['promo_interval'].fillna('', inplace=True)


        df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 1 if x['month_map'] in x['promo_interval'].split(',')
                                    else 0 ,axis=1)

        
        # competiton
        df1['competition_open_since_month'] = df1['competition_open_since_month'].astype( int64 )
        df1['competition_open_since_year'] = df1['competition_open_since_year'].astype( int64 )

        # promo2
        df1['promo2_since_week'] = df1['promo2_since_week'].astype( int64 )
        df1['promo2_since_year'] = df1['promo2_since_year'].astype( int64 )
        
        return df1
    
    def feature_engineering( self, df2):

        # Derivando datas como variáveis categóricas
        # year
        df2['year'] = df2['date'].dt.year

        # month
        df2['month'] = df2['date'].dt.month

        # day
        df2['day'] = df2['date'].dt.day

        # week of year
        df2['week_of_year'] = df2['date'].dt.weekofyear

        # year week
        df2['year_week'] = df2['date'].dt.strftime( '%Y-%W' )

        # competition since

        # criando uma única feature para dizer a data de início da promoção
        df2['competition_since'] = df2.apply( lambda x: datetime.datetime( year=x['competition_open_since_year'], month=x['competition_open_since_month'],day=1 ), axis=1 )

        # calcula o timedelta do início da promoção para a data do registro
        df2['competition_time_month'] = ( ( df2['date'] - df2['competition_since'] )/30 ).apply( lambda x: x.days ).astype( int64 )

        # promo since

        # cria uma str com o formato da data
        df2['promo_since'] = df2['promo2_since_year'].astype( str ) + '-' + df2['promo2_since_week'].astype( str )

        # transforma a str em data
        df2['promo_since'] = df2['promo_since'].apply( lambda x: datetime.datetime.strptime( x + '-1', '%Y-%W-%w' ) - datetime.timedelta( days=7 ) )

        # calcula o timedelta da promoção em semanas
        df2['promo_time_week'] = ( ( df2['date'] - df2['promo_since'] )/7 ).apply( lambda x: x.days ).astype( int64 )



        # competition strenght - indica de forma mais direta a interferência da competição

        df2['competition_str'] = 1/df2['competition_distance']


        ## PASSO 03 - FILTRAGEM DE VARIÁVEIS


        # Exclui linhas quando as lojas estavam fechadas ou não houve vendas
        df2 = df2[(df2['open'] != 0)]

        ## Feature Selection

        #Excluindo features que não fazem sentido para modelagem
        cols_drop = ['open', 'promo_interval', 'month_map', 'competition_open_since_year',
                     'competition_open_since_month', 'competition_since', 'promo_since', 'promo2_since_year',
                     'promo2_since_week', 'competition_distance' ]
        df2 = df2.drop( cols_drop, axis=1 )

        return df2
    
    def data_preparation(self, df5):
    
        ## Rescaling

        # competition_str
        df5['competition_str'] = self.encoding_competition_str.fit_transform(df5[['competition_str']].values)

        # competition time_month
        df5['competition_time_month'] = self.encoding_competition_time_month.fit_transform(df5[['competition_time_month']])

        # promo_time_week
        df5['promo_time_week'] = self.encoding_promo_time_week.fit_transform(df5[['promo_time_week']])

        # year
        df5['year'] = self.encoding_year.fit_transform(df5[['year']])


        ## Encoding

        # state_holiday - One Hot Encoding
        df5 = pd.get_dummies(df5, prefix=['state_holiday'], columns=['state_holiday'])

        # store_type - Label Encoding
        le = LabelEncoder()
        df5['store_type'] = self.encoding_store_type.fit_transform(df5['store_type'])

        # assortment - Ordinal Encoding
        assortment_dict = {'a':1,'b':2,'c':3}
        df5['assortment'] = df5['assortment'].map(assortment_dict)

        ## Transformations

        ### Nature transformation for cyclical variables

        # day of week
        df5['day_of_week_sin'] = df5['day_of_week'].apply(lambda x : np.sin(x*(2.*np.pi/7)))
        df5['day_of_week_cos'] = df5['day_of_week'].apply(lambda x : np.cos(x*(2.*np.pi/7)))

        # month
        df5['month_sin'] = df5['month'].apply(lambda x : np.sin(x*(2.*np.pi/12)))
        df5['month_cos'] = df5['month'].apply(lambda x : np.cos(x*(2.*np.pi/12)))

        # day
        df5['day_sin'] = df5['day'].apply(lambda x : np.sin(x*(2.*np.pi/30)))
        df5['day_cos'] = df5['day'].apply(lambda x : np.cos(x*(2.*np.pi/30)))

        # week of year
        df5['week_of_year_sin'] = df5['week_of_year'].apply(lambda x : np.sin(x*(2.*np.pi/52)))
        df5['week_of_year_cos'] = df5['week_of_year'].apply(lambda x : np.cos(x*(2.*np.pi/52)))
        
        cols_selected_boruta = ['store',
                            'promo',
                            'store_type',
                            'assortment',
                            'promo2',
                            'competition_time_month',
                            'promo_time_week',
                            'competition_str',
                            'day_of_week_sin',
                            'day_of_week_cos',
                            'month_cos',
                            'day_sin',
                            'day_cos',
                            'week_of_year_cos'
                           ]
        
        return df5[cols_selected_boruta]

    def get_prediction(self, model, original_data, test_data):
        #prediction
        pred = model.predict(test_data)
        
        # join pred with original data
        original_data['prediction'] = np.empm1(pred)
                  
        return original_data.to_json(orient='records', date_format = 'iso')
                  

## API Handler

In [7]:
import pandas as pd
import pickle
from flask import Flask, request, Response
from rossmann.Rossmann import Rossmann

#loading model
model = pickle.load(open(r'C:\Users\06564176686\repos\Store-Sales-Prediction\model\model_rossmann.pkl', 'rb'))

# initializa API
app = Flask(__name__)

@app.route('/rossmann/predict', methods = ['POST'])
def rossmann_predict():
    test_json = request.get_json()
    
    if test_json:
        # unique example
        if isinstance(test_json, dict):
            test_raw = pd.DataFrame(test_json, index=[0])
        # multiple examples
        else:
             test_raw = pd.DataFrame(test_json, columns = test_json[0].keys())
                
        # Instantiate Rossmann class
        pipeline = Rossmann()
                
        # data cleaning
        df1 = pipeline.data_cleaning(test_raw)
        
        # feature engineering
        df2 = pipeline.feature_engineering(df1)
            
        # data preparation
        df3 = pipeline.datapreparation(df2)
        
        # prediction
        df_response = pipeline.get_prediction(model, test_raw, df3)
        
        return df_response
        
    else:
        return Response('{}', status=200, mimetype='aookication/json')

if __name__ == 'main':
    app.run('0.0.0.0')
    

## API Tester

In [14]:
import requests
import json

# loading test dataset
df10 = pd.read_csv('data/test.csv', low_memory=False)

df_test = pd.merge(df10, df_store_raw, how='left', on='Store')

# choose store for prediction
df_test = df_test[df_test['Store']==34]

# remove closed days
df_test = df_test[df_test['Open']!=0]
df_test = df_test[~df_test['Open'].isnull()]
df_test = df_test.drop('Id', axis=1)

# convert DataFrame to json
data = json.dumps( df_test.to_dict( orient = 'records' ))



In [15]:
# API Call
url = 'http://0.0.0.0:5000/rossmann/predict'
header = {'Content-type': 'application/json'}
data = data

r = requests.post( url, data=data, headers=header)
print("Status Code {}.format(r.status_code)")

ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=5000): Max retries exceeded with url: /rossmann/predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000026A803B1580>: Failed to establish a new connection: [WinError 10049] O endereço solicitado não é válido no contexto'))

In [16]:
d1 = pd.DataFrame(r.json(), columns = r.json()[0].keys())

NameError: name 'r' is not defined

In [ ]:
d1.head()